In [1]:
import zipfile

with zipfile.ZipFile("walmart-10k-sales-datasets.zip", 'r') as zip_ref:
    zip_ref.extractall("unzipped_data")

print("Unzipped successfully!")

Unzipped successfully!


In [2]:
import pandas as pd

# Load the raw dataset
df = pd.read_csv('unzipped_data/Walmart.csv', encoding_errors='ignore')


# --- DATA CLEANING ---

#removing dollar sign and changing data type from str to float
df['unit_price'] = df['unit_price'].str.replace('$','', regex = False).astype(float)

# Drop duplicates
df.drop_duplicates(inplace=True)

# Fill missing numeric values with median
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill missing categorical values with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Adding new column Revenue
# Example: Revenue = unit price * quantity

if {'unit_price',  'profit_margin', 'quantity'}.issubset(df.columns):
    df['Revenue'] = df['unit_price'] * df['quantity']
    df['Total_profit'] = df['Revenue'] * df['profit_margin']
# Example: Revenue per unit
if {'Revenue', 'quantity'}.issubset(df.columns):
    df['Revenue_per_Unit'] = df['Revenue'] / df['quantity']

# Example: High/Low profit category
if 'Total_profit' in df.columns:
    median_pf = df['Total_profit'].median()
    df['Profit_Category'] = pd.cut(df['Total_profit'],
                                   bins=[-float('inf'), 0, median_pf, float('inf')],
                                   labels=['Loss', 'Low Profit', 'High Profit'])

# SAVE CLEAN DATA SAFELY 
df.to_csv('walmart_clean_data.csv', index=False)

print(f" Cleaned dataset saved to: ")

 Cleaned dataset saved to: 


In [8]:
import pymysql #works as adapter
from sqlalchemy import create_engine
from config import CONNECTION_STRING 

engine_mysql = create_engine(CONNECTION_STRING)
try:
    engine_mysql
    print("Yes")
except:
    print("No")


# Load a DataFrame into MySQL
df = pd.read_csv("Walmart.csv")
df.to_sql(name ='walmart', con = engine_mysql, if_exists = 'replace', index=False)

Yes


10051

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define the correct order of weekdays
week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Your SQL query (use the fixed version)
query2 = """
SELECT 
    branch,
    DAYNAME(date) AS day_name,
    AVG(total_profit) AS avg_profit
FROM walmart
GROUP BY branch, DAYNAME(date)
ORDER BY branch, avg_profit DESC;
"""

# Read data
df2 = pd.read_sql(query2, engine_mysql)

# Make 'day_name' categorical with the correct order
df2['day_name'] = pd.Categorical(df2['day_name'], categories=week_order, ordered=True)

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=df2, x='day_name', y='avg_profit', hue='branch')
plt.title("Average Profit by Day & Branch")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'total_profit' in 'field list'")
[SQL: 
SELECT 
    branch,
    DAYNAME(date) AS day_name,
    AVG(total_profit) AS avg_profit
FROM walmart
GROUP BY branch, DAYNAME(date)
ORDER BY branch, avg_profit DESC;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)